# Cheap Experimentation

This notebook contains scratch work for visualizing the equivariant activations and quantifying the embedding error when using this "Cheap" approach.

In [ ]:
import os
import cv2
import numpy as np
import torch
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
import matplotlib.pyplot as plt
from typing import Optional
from small_gg import SmallGGAutoencoder
from codec import Chunker, custom_load_img
import random

In [ ]:
def show(chunk):
    """
    Given a 2-dimensional greyscale rep of an image, show it
    """
    plt.imshow(chunk, cmap="gray", vmin=0.0, vmax=1.0)
    plt.axis("off")
    plt.show()

def show2(chunk1, chunk2):
    """
    Given two 2-dimensional grayscale representations of images, show them side by side.
    """
    fig, axes = plt.subplots(1, 2)
    axes[0].imshow(chunk1.squeeze(), cmap="gray", vmin=0.0, vmax=1.0)
    axes[0].axis("off")
    axes[1].imshow(chunk2.squeeze(), cmap="gray", vmin=0.0, vmax=1.0)
    axes[1].axis("off")
    plt.show()

In [ ]:
img = custom_load_img("frames/basic/i/frame_0.png")
chunks = [chunk for chunk in Chunker(img).to_chunks()]

In [ ]:
def generate_simple_test(chunk, shift):
    """
    Given a chunk (16x16 ndarray) and shift (tuple[int, int]) produce a
    48x48 ndarray with the 16x16 in the center, offset by shift
    """
    result = np.zeros((48, 48), dtype=chunk.dtype)
    result[16 + shift[1] : 16 + shift[1] + 16, 16 + shift[0] : 16 + shift[0] + 16] = chunk
    return result

In [ ]:
shift = (1, 1)
chunk = random.choice(chunks)
test = generate_simple_test(chunk.detach().numpy().reshape((16, 16)), shift)
post_chunk = test[16 : 32, 16 : 32]
post_chunk = torch.from_numpy(post_chunk).unsqueeze(0)
print("Visualizing test case...")
show2(chunk.detach().numpy().reshape((16, 16)), post_chunk)

model = SmallGGAutoencoder.load_model("autoencoders/gen3_200.pt")
og_prelude, og_bed = model.embed_with_prelude(chunk)
correct_prelude, correct_bed = model.embed_with_prelude(post_chunk)
print("")
print("Visualizing actual prelude difference...")
show2(og_prelude.detach().numpy(), correct_prelude.detach().numpy())

prelude_size = 16
naive_prelude = np.zeros((prelude_size, prelude_size), dtype=np.float32)
relevant = og_prelude.detach().numpy().reshape((prelude_size, prelude_size))
naive_prelude[shift[1] : prelude_size, shift[0] : prelude_size] = relevant[0 : prelude_size - shift[1], 0 : prelude_size - shift[0]]
print(correct_prelude[0,0,0])

print("Showing naive difference")
correct_clean = correct_prelude.detach().numpy().reshape((prelude_size, prelude_size))
show(np.abs(naive_prelude - correct_clean))


In [ ]:
img = custom_load_img("frames/basic/i/frame_0.png")
chunks = [chunk for chunk in Chunker(img, custom_size=32).to_chunks()]

In [ ]:
def shift_chunk(chunk, shift):
    """
    Shift a chunk with out-of-bounds handling
    """
    shifted_chunk = np.zeros_like(chunk)
    
    # Determine the ranges for copying from the original chunk to the shifted chunk
    row_range = slice(max(0, -shift[0]), min(chunk.shape[0], chunk.shape[0] - shift[0]))
    col_range = slice(max(0, -shift[1]), min(chunk.shape[1], chunk.shape[1] - shift[1]))
    
    # Determine the ranges for pasting into the shifted chunk
    new_row_range = slice(max(0, shift[0]), min(chunk.shape[0], chunk.shape[0] + shift[0]))
    new_col_range = slice(max(0, shift[1]), min(chunk.shape[1], chunk.shape[1] + shift[1]))
    
    # Perform the shift and handle out-of-bounds
    shifted_chunk[new_row_range, new_col_range] = chunk[row_range, col_range]
    
    return shifted_chunk

def simulate_diff(chunk, frac_random=0.25):
    """
    Slightly too lazy to actually get the chunks from consecutive frames so
    just going to add in some randomness
    Will replace `frac_random` of the entries with 1/64 * random number in [0, 63]
    NOTE: Does NOT modify chunk returns a new chunk
    """
    num_set = int((chunk.shape[0] * chunk.shape[1]) * frac_random)
    result = chunk.copy()
    for _ in range(num_set):
        row = random.randint(0, chunk.shape[0] - 1)
        col = random.randint(0, chunk.shape[1] - 1)
        mult = sum([random.randint(-1, 1) for _ in range(3)])
        new_val = result[row, col] + 1.0 / 64.0 * mult
        result[row, col] = max(min(new_val, 1.0), 0.0)
    return result


In [ ]:
# ix = random.randint(0, len(chunks) - 1)
ix = 65
chunk_tens = chunks[ix]
chunk_np = chunk_tens.detach().numpy().reshape((32, 32))
chunk = chunks[ix].detach().numpy().reshape((32, 32))
shift = (-2, 3)
shifted = shift_chunk(chunk, shift)
diff = simulate_diff(shifted)
print("Will be running test on the following chunks")
show2(chunk, diff)

In [ ]:
def box_show(chunk, boxes=None, offset=(-0.5, -0.5)):
    """
    Given a 2-dimensional greyscale rep of an image, show it
    """
    plt.imshow(chunk, cmap="gray", vmin=0.0, vmax=1.0)
    
    if boxes:
        for box, color in boxes:
            top_left, bottom_right = box
            # Apply offset to coordinates
            top_left = (top_left[0] + offset[0], top_left[1] + offset[1])
            bottom_right = (bottom_right[0] + offset[0], bottom_right[1] + offset[1])
            # Plot the box
            plt.plot([top_left[0], bottom_right[0], bottom_right[0], top_left[0], top_left[0]],
                     [top_left[1], top_left[1], bottom_right[1], bottom_right[1], top_left[1]], color=color, linewidth=4)
    
    plt.axis("off")
    plt.show()

box_show(chunk, [(((16, 0), (32, 16)),"red"), (((13, 2), (29, 18)),"pink")])
box_show(diff, [(((16, 0), (32, 16)),"pink"), ])

In [ ]:
chunk_size = 16
import math

def dx_to_top_left(dx):
    img_height = 32
    chunk_height = int(math.ceil(img_height / chunk_size))
    x = (dx // chunk_height) * chunk_size
    y = (dx % chunk_height) * chunk_size
    return (x, y)

og_preludes = np.zeros_like(chunk)
correct_preludes = np.zeros_like(shifted)

for dx, c in enumerate(Chunker(chunk).to_chunks()):
    og_prelude, _ = model.embed_with_prelude(c)
    x, y = dx_to_top_left(dx)
    og_preludes[y : y + chunk_size, x : x + chunk_size] = og_prelude.detach().numpy()

for dx, c in enumerate(Chunker(shifted).to_chunks()):
    shifted_prelude, _ = model.embed_with_prelude(c)
    x, y = dx_to_top_left(dx)
    correct_preludes[y : y + chunk_size, x : x + chunk_size] = shifted_prelude.detach().numpy()

box_show(og_preludes, [(((16, 0), (32, 16)),"red"), (((13, 2), (29, 18)),"pink")])
box_show(correct_preludes, [(((16, 0), (32, 16)),"pink"), ])

In [ ]:
guessed = og_preludes[2 : 18, 13 : 29]
correct = correct_prelude[0 : 16, 16 : 32]
acc_diff = guessed - correct
show()